# Traffic Light Detection using YOLOv8 on the LISA Dataset (Combined Day/Night) - Version with Augmentation

Implementation and training of the YOLOv8 model.

## 1. Imports and Basic Configuration

In [ ]:
from ultralytics import YOLO
import os
import yaml
import torch
import json

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

YOLO_CLASS_NAMES = ['go', 'stop', 'warning', 'off'] 
NUM_CLASSES_YOLO = len(YOLO_CLASS_NAMES)

YOLO_DATA_ROOT_COMBINED = "../dataset/lisa_yolo_formatted/" 
OUTPUT_DIR_YOLO = "../results/yolo/"
os.makedirs(OUTPUT_DIR_YOLO, exist_ok=True)

YOLO_MODEL_VARIANT = 'yolov8s.pt'
IMAGE_SIZE_YOLO = 640
EPOCHS_YOLO = 10             
BATCH_SIZE_YOLO = 4         
LEARNING_RATE_YOLO = 0.001 
WEIGHT_DECAY_YOLO = 0.0005 
OPTIMIZER_YOLO = 'AdamW'
PATIENCE_YOLO = 20
SAVE_PERIOD_YOLO = -1

APPLY_AUGMENTATIONS = True

AUG_HSV_H = 0.015
AUG_HSV_S = 0.7
AUG_HSV_V = 0.4
AUG_DEGREES = 10.0
AUG_TRANSLATE = 0.1
AUG_SCALE = 0.1
AUG_SHEAR = 5.0
AUG_PERSPECTIVE = 0.0
AUG_FLIPUD = 0.0
AUG_FLIPLR = 0.5
AUG_MOSAIC = 1.0
AUG_MIXUP = 0.1
AUG_COPY_PASTE = 0.1

USE_SUBSET_FOR_YOLO_TRAINING_INFO = True 

training_args_log = {
    "model_name": YOLO_MODEL_VARIANT,
    "image_size": IMAGE_SIZE_YOLO,
    "epochs": EPOCHS_YOLO,
    "batch_size": BATCH_SIZE_YOLO,
    "learning_rate": LEARNING_RATE_YOLO,
    "weight_decay": WEIGHT_DECAY_YOLO,
    "optimizer": OPTIMIZER_YOLO,
    "patience_early_stopping": PATIENCE_YOLO,
    "save_period_checkpoint": SAVE_PERIOD_YOLO,
    "apply_augmentations": APPLY_AUGMENTATIONS,
    "augmentation_params" : {
        "hsv_h": AUG_HSV_H if APPLY_AUGMENTATIONS else 0.0,
        "hsv_s": AUG_HSV_S if APPLY_AUGMENTATIONS else 0.0,
        "hsv_v": AUG_HSV_V if APPLY_AUGMENTATIONS else 0.0,
        "degrees": AUG_DEGREES if APPLY_AUGMENTATIONS else 0.0,
        "translate": AUG_TRANSLATE if APPLY_AUGMENTATIONS else 0.0,
        "scale": AUG_SCALE if APPLY_AUGMENTATIONS else 0.0,
        "shear": AUG_SHEAR if APPLY_AUGMENTATIONS else 0.0,
        "perspective": AUG_PERSPECTIVE if APPLY_AUGMENTATIONS else 0.0,
        "flipud": AUG_FLIPUD if APPLY_AUGMENTATIONS else 0.0,
        "fliplr": AUG_FLIPLR if APPLY_AUGMENTATIONS else 0.0,
        "mosaic": AUG_MOSAIC if APPLY_AUGMENTATIONS else 0.0,
        "mixup": AUG_MIXUP if APPLY_AUGMENTATIONS else 0.0,
        "copy_paste": AUG_COPY_PASTE if APPLY_AUGMENTATIONS else 0.0
    },
    "yolo_class_names": YOLO_CLASS_NAMES,
    "num_classes_yolo": NUM_CLASSES_YOLO,
    "yolo_data_root_combined": os.path.abspath(YOLO_DATA_ROOT_COMBINED),
    "use_subset_info": USE_SUBSET_FOR_YOLO_TRAINING_INFO
}

augmentation_status_str = "aug" if APPLY_AUGMENTATIONS else "no_aug"
args_filename = f'training_arguments_yolov8_{augmentation_status_str}.json'
args_save_path = os.path.join(OUTPUT_DIR_YOLO, args_filename)
try:
    with open(args_save_path, 'w') as f:
        json.dump(training_args_log, f, indent=4)
    print(f"YOLOv8 training arguments saved to {args_save_path}")
except Exception as e:
    print(f"Error saving YOLOv8 training arguments: {e}")

## 2. Preparing the `dataset.yaml` Configuration File for YOLO (Combined Data)

In [ ]:
def create_yolo_dataset_yaml_combined(yolo_data_root, num_classes, class_names_list):
    yaml_file_path = os.path.join(yolo_data_root, f"dataset_combined.yaml")

    train_images_path = os.path.join(yolo_data_root, 'images', 'train')
    val_images_path = os.path.join(yolo_data_root, 'images', 'val')

    if not (os.path.isdir(os.path.abspath(train_images_path)) and os.path.isdir(os.path.abspath(val_images_path))):
        print(f"CRITICAL ERROR: Required train/val image folders are missing in {os.path.abspath(yolo_data_root)}")
        print("Ensure that `lisa_dataset_preparation.ipynb` correctly created the structure for YOLO.")
        print(f"Checked: {os.path.abspath(train_images_path)} (exists: {os.path.isdir(os.path.abspath(train_images_path))})")
        print(f"Checked: {os.path.abspath(val_images_path)} (exists: {os.path.isdir(os.path.abspath(val_images_path))})")
        return None
        
    yaml_content = {
        'path': os.path.abspath(yolo_data_root),
        'train': 'images/train',
        'val': 'images/val',
        'nc': num_classes,
        'names': class_names_list
    }
    
    try:
        with open(yaml_file_path, 'w') as f:
            yaml.dump(yaml_content, f, sort_keys=False, indent=2)
        print(f"Generated combined YAML file for YOLO: {yaml_file_path}")
        return yaml_file_path
    except Exception as e:
        print(f"Error creating YAML file {yaml_file_path}: {e}")
        return None

dataset_yaml_path_combined = create_yolo_dataset_yaml_combined(YOLO_DATA_ROOT_COMBINED, NUM_CLASSES_YOLO, YOLO_CLASS_NAMES)

## 3. YOLOv8 Training Loop (Combined Data)

In [ ]:
if dataset_yaml_path_combined:
    print(f"\n--- Training YOLOv8 on combined data (Day/Night) with augmentation: {APPLY_AUGMENTATIONS} ---")
    
    model = YOLO(YOLO_MODEL_VARIANT)
    
    experiment_name = f"yolov8_{YOLO_MODEL_VARIANT.split('.')[0]}_combined_{augmentation_status_str}_exp"
    
    train_params = {
        'data': dataset_yaml_path_combined,
        'epochs': EPOCHS_YOLO,
        'batch': BATCH_SIZE_YOLO,
        'imgsz': IMAGE_SIZE_YOLO,
        'project': OUTPUT_DIR_YOLO,
        'name': experiment_name,
        'exist_ok': True,
        'patience': PATIENCE_YOLO,
        'optimizer': OPTIMIZER_YOLO,
        'lr0': LEARNING_RATE_YOLO,
        'weight_decay': WEIGHT_DECAY_YOLO,
        'save_period': SAVE_PERIOD_YOLO,
        'device': DEVICE,
        'verbose': True,
        'seed': 42,
        'plots': True
    }

    if APPLY_AUGMENTATIONS:
        train_params.update({
            'augment': True,
            'hsv_h': AUG_HSV_H,
            'hsv_s': AUG_HSV_S,
            'hsv_v': AUG_HSV_V,
            'degrees': AUG_DEGREES,
            'translate': AUG_TRANSLATE,
            'scale': AUG_SCALE,
            'shear': AUG_SHEAR,
            'perspective': AUG_PERSPECTIVE,
            'flipud': AUG_FLIPUD,
            'fliplr': AUG_FLIPLR,
            'mosaic': AUG_MOSAIC, 
            'mixup': AUG_MIXUP,
            'copy_paste': AUG_COPY_PASTE
        })
    else:
        train_params.update({
            'augment': False,
            'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0,
            'degrees': 0.0, 'translate': 0.0, 'scale': 0.0, 'shear': 0.0,
            'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.0, 
            'mosaic': 0.0, 'mixup': 0.0, 'copy_paste': 0.0
        })

    try:
        print(f"Starting training with the following parameters: {train_params}")
        model.train(**train_params)
        print(f"\nYOLOv8 training ({augmentation_status_str}) completed.")
        print(f"Results saved in directory: {os.path.join(OUTPUT_DIR_YOLO, experiment_name)}")
        print(f"The 'results.csv' file with metrics should be in the above directory.")
    except Exception as e:
        print(f"Error during YOLOv8 training: {e}")

else:
    print("Failed to create dataset_combined.yaml file for YOLO. Training skipped.")

print("\n--- YOLOv8 script completed ---")